In [1]:
import argparse
import logging
import sys
import time

from tf_pose import common
import cv2
import numpy as np
from tf_pose.estimator import TfPoseEstimator
from tf_pose.networks import get_graph_path, model_wh

import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [2]:
# init logger
logger = logging.getLogger('TfPoseEstimatorRun')
logger.handlers.clear()
logger.setLevel(logging.ERROR)
ch = logging.StreamHandler()
ch.setLevel(logging.WARNING)
formatter = logging.Formatter('[%(asctime)s] [%(name)s] [%(levelname)s] %(message)s')
ch.setFormatter(formatter)
logger.addHandler(ch)

In [3]:
class MyArgs():
    def __init__(self):
        self.image = './images/p1.jpg'
        self.model = 'mobilenet_thin'
        self.resize = '0x0'
        self.resize_out_ratio = 4.0
args = MyArgs()

In [4]:
w, h = model_wh(args.resize)
if w == 0 or h == 0:
    e = TfPoseEstimator(get_graph_path(args.model), target_size=(432, 368))
else:
    e = TfPoseEstimator(get_graph_path(args.model), target_size=(w, h))

[2020-11-11 08:28:05,272] [TfPoseEstimator] [INFO] loading graph from /home/jovyan/work/fall-detection/tf-pose-estimation/models/graph/mobilenet_thin/graph_opt.pb(default size=432x368)
2020-11-11 08:28:05,272 INFO loading graph from /home/jovyan/work/fall-detection/tf-pose-estimation/models/graph/mobilenet_thin/graph_opt.pb(default size=432x368)


TfPoseEstimator/MobilenetV1/Conv2d_0/weights
TfPoseEstimator/image
TfPoseEstimator/MobilenetV1/Conv2d_0/Conv2D
TfPoseEstimator/MobilenetV1/Conv2d_0/Conv2D_bn_offset
TfPoseEstimator/MobilenetV1/Conv2d_0/BatchNorm/FusedBatchNorm
TfPoseEstimator/MobilenetV1/Conv2d_0/Relu
TfPoseEstimator/MobilenetV1/Conv2d_1_depthwise/depthwise_weights
TfPoseEstimator/MobilenetV1/Conv2d_1_pointwise/weights
TfPoseEstimator/MobilenetV1/Conv2d_1_depthwise/depthwise
TfPoseEstimator/MobilenetV1/Conv2d_1_pointwise/Conv2D
TfPoseEstimator/MobilenetV1/Conv2d_1_pointwise/Conv2D_bn_offset
TfPoseEstimator/MobilenetV1/Conv2d_1_pointwise/BatchNorm/FusedBatchNorm
TfPoseEstimator/MobilenetV1/Conv2d_1_pointwise/Relu
TfPoseEstimator/MobilenetV1/Conv2d_2_depthwise/depthwise_weights
TfPoseEstimator/MobilenetV1/Conv2d_2_pointwise/weights
TfPoseEstimator/MobilenetV1/Conv2d_2_depthwise/depthwise
TfPoseEstimator/MobilenetV1/Conv2d_2_pointwise/Conv2D
TfPoseEstimator/MobilenetV1/Conv2d_2_pointwise/Conv2D_bn_offset
TfPoseEstimator/M

In [5]:
def handleImage(image_path, window_size = (15, 15)):
    global e
    global logger
    image = common.read_imgfile(image_path, None, None)
    if image is None:
        logger.error('Image can not be read, path=%s' % args.image)
        #sys.exit(-1)
        return

    t = time.time()
    
    # 高斯模糊
    image = cv2.GaussianBlur(image, window_size, 0)
    # 轉換影像為數值資料
    humans = e.inference(image, resize_to_default=(w > 0 and h > 0), upsample_size=args.resize_out_ratio)
    
    elapsed = time.time() - t
    logger.info('inference image: %s in %.4f seconds.' % (args.image, elapsed))
    # image = TfPoseEstimator.draw_humans(image, humans, imgcopy=False)

    try:
        return humans
    except Exception as e:
        logger.warning('matplitlib error, %s' % e)
        cv2.waitKey()
        return None

In [6]:
import os
import numpy as np
import csv
import pandas as pd
import functools
import operator

point_nums = 17

def listFlat(l):
    return functools.reduce(operator.iconcat, l, [])

def handleHumanData(h):
    tmp_data = []
    for i in range(0, point_nums + 1):
        if i in h.body_parts.keys():
            p = h.body_parts[i]
            x = p.x
            y = p.y
            score = p.score
        else:
            x = np.nan
            y = np.nan
            score = np.nan
        tmp_data.append({ 'x': x, 'y': y, 'score': score })
    df = pd.DataFrame(tmp_data)
    df['x'] = df['x'] - df['x'].mean()
    df['y'] = df['y'] - df['y'].mean()
    df['x'].fillna(value=0, inplace=True)
    df['y'].fillna(value=0, inplace=True)
    df['score'].fillna(value=df['score'].mean(), inplace=True)
    return listFlat(df.values)

In [7]:
# 將影像人工分類後，將各個類別全都放到一個資料夾底下，這個函數用來將整個資料夾轉成一個csv檔，來供後續訓練。
# 例如：fall資料夾底下全部只放跌倒的照片，之後把fall轉成fall.csv。
#
# @save_to: 輸出的csv檔路徑
#
# @root_paths: 要轉換的資料夾清單。會轉換資料夾底下所有影像檔案。
# 注意，這邊的程式沒有去判斷檔案是否是影像檔，因此要確保轉換的資料夾底下全部都是影像檔案。
def handleDataset(save_to, root_paths, window_size = (15, 15)):
    start_time = time.time()
    
    datas = [[]]
    cnt = 0
    zeroh_nums = 0
    
    for i in range(0, point_nums + 1):
        datas[0].extend(['x' + str(i), 'y' + str(i), 'score' + str(i)])
    for r in root_paths:
        for path, dir_list, file_list in os.walk(r):
            for file_name in file_list:
                _file = os.path.join(path, file_name)
                # print("handle:", _file, ' ...')
                h = handleImage(_file, window_size)
                if len(h) == 0:
                    # print('number of human is zero: ' + _file)
                    zeroh_nums = zeroh_nums + 1
                    continue
                cnt = cnt + 1
                h = h[0]
                datas.append(handleHumanData(h))
    if save_to is not None:        
        with open(save_to, 'w', newline='') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerows(datas)
    p = time.time() - start_time
    
    rate = cnt * 100 / (cnt + zeroh_nums)
    print('window size of Gaussian Blur:', window_size)
    print('zeㄐo/images: %d/%d | %.2f%%' % (zeroh_nums, (cnt + zeroh_nums), rate))
    print('finish in %.4f seconds.' % p)
    return rate

In [10]:
ls = [
    './mdata/images/bed/BedTest/stand',
    './mdata/images/bed/BedTest/squatting',
    './mdata/images/bed/BedTest/sitting'
]
handleDataset('./mdata/rows/BedTest-no-fall.csv', ls, (3, 3))

window size of Gaussian Blur: (3, 3)
zeor/images: 1/421 | 99.76%
finish in 28.5053 seconds.


99.7624703087886

In [9]:
# rates = []
# for i in range(1, 50, 2):
#     r = handleDataset(None, ['./mdata/images/bed/stand'], (i, i))
#     rates.append(r)